# Milestone Report: Predicting the All-NBA Teams
### Background
At the end of the NBA regular season, the Associated Press honors the elite few with an award called making the “All-NBA” team. The All-NBA team is similar to making a theoretical All-Star game of a starting 5 Roster for who is the best at each of the respected positions (Point Guard, Shooting Guard, Small Forward, Power Forward, Center). There are 3 sets of All-NBA teams, known as: First, Second, and Third Team. Equivalent to Gold, Metal and Bronze at the Olympics. Besides for winning the Most Valuable Player award (MVP), making the All-NBA team is considered to be among the most prestigious awards for weighing one’s personal performance and has a large impact when incentivizing bonuses in a player’s contract, providing evidence to demand a larger contract salary when contract negotiations ensue in the offseason and is a standout optic when building one’s resume for making the basketball hall of fame at the end of a player’s career. People tend to measure All-NBA notoriety when deciding to vote for that player or not. 
The Associated Press are experts in the world of Journalism, but when it comes to voting, it is solely up to them. While they may be respected experts in this field and ultimately are the people who decide the fate of a player, when it comes to how they choose to lobby for that player to make the All-NBA team, not much is really known. Do they do their due diligence when evaluating that player and measure the all-around depth, or rather do they simply look at the basic stats and popularity throughout the season. Hopefully analyzing data from a Rating standpoint which provides a better all-around assessment of the NBA’s top performers can not only help predict who should make the All-NBA team, but also provide a more accurate assessment to give those who are deserving of being recognized that may not have necessarily gotten the chance because their personal brand may not have been as popular.


### The Data
The data was obtained from 3 popular basketball websites that provide real-time updates of NBA player stats and from decades of previous seasons. Why I chose to use 4 sites is because not one site was able to provide all of the variables that I aimed to collect throughout the season, so I combined each site that was able to do so. I did not conduct any web scraping for this project. Each season’s data is contained in a separate .csv file.
I have primarily conducted research on the datasets for this past regular season, used a formula of relevant and significant variables to create 2 new variables called “Playmaker Rating” and “Scorer Rating” to help predict the “best of the best,” based on these metrics; who should be awarded to the All-NBA first team, solely on the metrics and nothing further. I then use these Ratings to analyze the last 16 regular seasons to see how consistent the top playmaker and scorer Players match up with who was selected to the first, second and third All NBA teams

### Important Fields
The datasets each have roughly 90 variables. For the purpose of this project, the most important fields will be those relating to the conventional player stats (PPG, AST, REB, Usage percent) and the unconventional player stats (Playmaker and Scorer Rating, aPPG, uPPG). 

### Limitations to the Data
When wrangling the data from the various sites, some players were labeled a bit differently from one to another. For example, JJ Reddick on one site would have been labeled as J.J Reddick on another site, JR Smith and J.R Smith, etc.... Other players had hyphenated last names on one site, where it may have differed on another. Overall, name structuring seemed to vary a bit. Over the course of 16 seasons there are many players in the NBA with an abbreviated name of some sort and due to case sensitivity and time efficiency purposes, it would have wasted time to match every individual player correctly. With that being said, this resulted in a few dozen players missing personal stats. 
Another limitation came up regarding the filtering of players to a specific position. For example, some players were labeled a broad position ie) “Guard, Forward, etc..” and not “Point Guard, Shooting Guard, Small Forward, Power Forward, etc…” Not specifying the exact position, they played may have mislabeled that player when they might have had a higher ranking as lets say, a shooting guard, but being put in the overall pool of all guards lower their chances of recognition.

### Cleaning & Wrangling
There was very minor wrangling that needed to be done for this data. I decided to make a csv file for each season, take the stats from each website and create a Vlookup on excel to match each variable to that individual player. I then imported the csv using pandas and worked from there to create the new variables based from the originals. 
To keep the data more manageable and more simple, I created new dataframes that eliminated the extraneous variables, creating new datasets with these variables for each season.


**Making a new database with the relevant variables**

df16_17v2 = df16_17[['Season','Player', 'Position', 'G_total_player', 'THREEPM_pg_player', 'TWOPM_pg_player', 
          'THREEFGMpercentAST_1_player', 'TWOFGMpercentAST_1_player', 'FTpercent_pg_1_player', 
          'FTA_pg_player', 'ORBpercent_1_player', 'Ortg_1_team', 'FGA_pg_player',
         'FGM_pg_player', 'FTA_pg_team', 'FGA_pg_team', 'ORB_pg_team', 'TO_pg_team', 'PTS_pg_team']
        ].drop_duplicates().reset_index(drop=True)

**Adding FG Missed/gm column to the dataframe2**

df16_17v2['FG_Missed_pg_player'] = df16_17v2['FGA_pg_player'] - df16_17v2['FGM_pg_player']

**Adding unassisted points per game columns to dataframe2**

df16_17v2['uPPG'] = ((3*df16_17v2['THREEPM_pg_player']* (1-df16_17v2['THREEFGMpercentAST_1_player']) + 
                      2*df16_17v2['TWOPM_pg_player']*
               (1-df16_17v2['TWOFGMpercentAST_1_player'])) / df16_17v2['G_total_player'])*100

**Adding assissted points per game column to the dataframe2**

df16_17v2['aPPG'] = ((3*df16_17v2['THREEPM_pg_player']*df16_17v2['THREEFGMpercentAST_1_player'] + 
                2*df16_17v2['TWOPM_pg_player']*df16_17v2['TWOFGMpercentAST_1_player']) / 
                     (2*df16_17v2['G_total_player']))*100


**Adding Possessions by team variable to the dataframe2**

df16_17v2['Poss_team'] = df16_17v2['FGA_pg_team'] + (0.44) * df16_17v2['FTA_pg_team'] - df16_17v2['ORB_pg_team'] + df16_17v2['TO_pg_team']

**Adding Team Points per Possession to the dataframe2**

df16_17v2['PTS_pposs_team'] = (df16_17v2['PTS_pg_team'] / (df16_17v2['FGA_pg_team']  + 0.44 * 
                                                           df16_17v2['FTA_pg_team']))*((df16_17v2['FGA_pg_team'] +
                                            0.44 * df16_17v2['FTA_pg_team']) / df16_17v2['Poss_team'])

**Finalizing the Scorer Rating Column with the proper variables**

df16_17v2['Scorer_Rating'] = (df16_17v2['uPPG'] + df16_17v2['aPPG'] - 
                            df16_17v2['FG_Missed_pg_player'])*(1-df16_17v2['ORBpercent_1_player'])* df16_17v2['Ortg_1_team'] + (
                            df16_17v2['FTpercent_pg_1_player']*(df16_17v2['FTA_pg_player']/(df16_17v2['FTA_pg_player']*0.44) - 
                            df16_17v2['PTS_pposs_team']))* (df16_17v2['FTA_pg_player']*0.44) 
                            


df16_17v5 = df16_17[['Season','Player', 'Position', 'G_total_player', 'PTS_pg_player', 'AST_pg_player', 'TOV_pg_player', 
          'FGA_pg_player', 'FTA_pg_player', 'USGpercent_1_player']
        ].drop_duplicates().reset_index(drop=True)

**Creating a Playmaker Rating variable**

df16_17v5['Playmaker_Rating'] = (df16_17v5['USGpercent_1_player']*(df16_17v5['PTS_pg_player'] + 
                                                2.26*df16_17v5['AST_pg_player'] - 
                                                df16_17v5['TOV_pg_player']) / (df16_17v5['FGA_pg_player'] + 
                                                0.44*df16_17v5['FTA_pg_player'] + df16_17v5['AST_pg_player'] + 
                                                df16_17v5['TOV_pg_player']))


## Scorer Rating

**2000-01 Scorer Rating**

df00_01 = NBA2000_2016v4[NBA2000_2016v4.Season == 2001]
df00_01.sort_values(by='Scorer_Rating', ascending=False)
df00_01v2 = df00_01.sort_values(['Position', 'Scorer_Rating'],ascending=[False,False])

PG00_01 = df00_01v2[df00_01.Position == 'PG']
SG00_01 = df00_01v2[df00_01.Position == 'SG']
SF00_01 = df00_01v2[df00_01.Position == 'SF']
PF00_01 = df00_01v2[df00_01.Position == 'PF']
C00_01 = df00_01v2[df00_01.Position == 'C']

PG00_01_starters = df00_01v2[df00_01.Position == 'PG'].head(1)
SG00_01_starters = df00_01v2[df00_01.Position == 'SG'].head(1)
SF00_01_starters = df00_01v2[df00_01.Position == 'SF'].head(1)
PF00_01_starters = df00_01v2[df00_01.Position == 'PF'].head(1)
C00_01_starters = df00_01v2[df00_01.Position == 'C'].head(1)

SR00_01 = pd.concat([PG00_01, SG00_01, SF00_01, PF00_01, C00_01])
SR00_01_starters = pd.concat([PG00_01_starters, SG00_01_starters, SF00_01_starters, PF00_01_starters, C00_01_starters])
#---------------------------------------------------------------------------------------------------------------------#

**2001-02 Scorer Rating**

df01_02 = NBA2000_2016v4[NBA2000_2016v4.Season == 2002]
df01_02.sort_values(by='Scorer_Rating', ascending=False)
df01_02v2 = df01_02.sort_values(['Position', 'Scorer_Rating'],ascending=[False,False])

PG01_02 = df01_02v2[df01_02.Position == 'PG']
SG01_02 = df01_02v2[df01_02.Position == 'SG']
SF01_02 = df01_02v2[df01_02.Position == 'SF']
PF01_02 = df01_02v2[df01_02.Position == 'PF']
C01_02 = df01_02v2[df01_02.Position == 'C']

PG01_02_starters = df01_02v2[df01_02.Position == 'PG'].head(1)
SG01_02_starters = df01_02v2[df01_02.Position == 'SG'].head(1)
SF01_02_starters = df01_02v2[df01_02.Position == 'SF'].head(1)
PF01_02_starters = df01_02v2[df01_02.Position == 'PF'].head(1)
C01_02_starters = df01_02v2[df01_02.Position == 'C'].head(1)

SR01_02 = pd.concat([PG01_02, SG01_02, SF01_02, PF01_02, C01_02])
SR01_02_starters = pd.concat([PG01_02_starters, SG01_02_starters, SF01_02_starters, PF01_02_starters, C01_02_starters])
#---------------------------------------------------------------------------------------------------------------------#

**2002-03 Scorer Rating**

df02_03 = NBA2000_2016v4[NBA2000_2016v4.Season == 2003]
df02_03.sort_values(by='Scorer_Rating', ascending=False)

df02_03v2 = df02_03.sort_values(['Position', 'Scorer_Rating'],ascending=[False,False])

PG02_03 = df02_03v2[df02_03.Position == 'PG']
SG02_03 = df02_03v2[df02_03.Position == 'SG']
SF02_03 = df02_03v2[df02_03.Position == 'SF']
PF02_03 = df02_03v2[df02_03.Position == 'PF']
C02_03 = df02_03v2[df02_03.Position == 'C']

PG02_03_starters = df02_03v2[df02_03.Position == 'PG'].head(1)
SG02_03_starters = df02_03v2[df02_03.Position == 'SG'].head(1)
SF02_03_starters = df02_03v2[df02_03.Position == 'SF'].head(1)
PF02_03_starters = df02_03v2[df02_03.Position == 'PF'].head(1)
C02_03_starters = df02_03v2[df02_03.Position == 'C'].head(1)

SR02_03 = pd.concat([PG02_03, SG02_03, SF02_03, PF02_03, C02_03])
SR02_03_starters = pd.concat([PG02_03_starters, SG02_03_starters, SF02_03_starters, PF02_03_starters, C02_03_starters])
#---------------------------------------------------------------------------------------------------------------------#

**2003-2004 Scorer Rating**

df03_04 = NBA2000_2016v4[NBA2000_2016v4.Season == 2004]
df03_04.sort_values(by='Scorer_Rating', ascending=False)
df03_04v2 = df03_04.sort_values(['Position', 'Scorer_Rating'],ascending=[False,False])

PG03_04 = df03_04v2[df03_04.Position == 'PG']
SG03_04 = df03_04v2[df03_04.Position == 'SG']
SF03_04 = df03_04v2[df03_04.Position == 'SF']
PF03_04 = df03_04v2[df03_04.Position == 'PF']
C03_04 = df03_04v2[df03_04.Position == 'C']

PG03_04_starters = df03_04v2[df03_04.Position == 'PG'].head(1)
SG03_04_starters = df03_04v2[df03_04.Position == 'SG'].head(1)
SF03_04_starters = df03_04v2[df03_04.Position == 'SF'].head(1)
PF03_04_starters = df03_04v2[df03_04.Position == 'PF'].head(1)
C03_04_starters = df03_04v2[df03_04.Position == 'C'].head(1)


SR03_04 = pd.concat([PG03_04, SG03_04, SF03_04, PF03_04, C03_04])
SR03_04_starters = pd.concat([PG03_04_starters, SG03_04_starters, SF03_04_starters, PF03_04_starters, C03_04_starters])
#---------------------------------------------------------------------------------------------------------------------#

**2004-2005 Scorer Rating**

df04_05 = NBA2000_2016v4[NBA2000_2016v4.Season == 2005]
df04_05.sort_values(by='Scorer_Rating', ascending=False)

df04_05v2 = df04_05.sort_values(['Position', 'Scorer_Rating'],ascending=[False,False])

PG04_05 = df04_05v2[df04_05.Position == 'PG']
SG04_05 = df04_05v2[df04_05.Position == 'SG']
SF04_05 = df04_05v2[df04_05.Position == 'SF']
PF04_05 = df04_05v2[df04_05.Position == 'PF']
C04_05 = df04_05v2[df04_05.Position == 'C']

PG04_05_starters = df04_05v2[df04_05.Position == 'PG'].head(1)
SG04_05_starters = df04_05v2[df04_05.Position == 'SG'].head(1)
SF04_05_starters = df04_05v2[df04_05.Position == 'SF'].head(1)
PF04_05_starters = df04_05v2[df04_05.Position == 'PF'].head(1)
C04_05_starters = df04_05v2[df04_05.Position == 'C'].head(1)


SR04_05 = pd.concat([PG04_05, SG04_05, SF04_05, PF04_05, C04_05])
SR04_05_starters = pd.concat([PG04_05_starters, SG04_05_starters, SF04_05_starters, PF04_05_starters, C04_05_starters])
#---------------------------------------------------------------------------------------------------------------------#

**2005-06 Scorer Rating**

df05_06 = NBA2000_2016v4[NBA2000_2016v4.Season == 2006]
df05_06.sort_values(by='Scorer_Rating', ascending=False)

df05_06v2 = df05_06.sort_values(['Position', 'Scorer_Rating'],ascending=[False,False])

PG05_06 = df05_06v2[df05_06.Position == 'PG']
SG05_06 = df05_06v2[df05_06.Position == 'SG']
SF05_06 = df05_06v2[df05_06.Position == 'SF']
PF05_06 = df05_06v2[df05_06.Position == 'PF']
C05_06 = df05_06v2[df05_06.Position == 'C']

PG05_06_starters = df05_06v2[df05_06.Position == 'PG'].head(1)
SG05_06_starters = df05_06v2[df05_06.Position == 'SG'].head(1)
SF05_06_starters = df05_06v2[df05_06.Position == 'SF'].head(1)
PF05_06_starters = df05_06v2[df05_06.Position == 'PF'].head(1)
C05_06_starters = df05_06v2[df05_06.Position == 'C'].head(1)

SR05_06 = pd.concat([PG05_06, SG05_06, SF05_06, PF05_06, C05_06])
SR05_06_starters = pd.concat([PG05_06_starters, SG05_06_starters, SF05_06_starters, PF05_06_starters, C05_06_starters])
#---------------------------------------------------------------------------------------------------------------------#

**2006-07 Scorer Rating**

df06_07 = NBA2000_2016v4[NBA2000_2016v4.Season == 2007]
df06_07.sort_values(by='Scorer_Rating', ascending=False)

df06_07v2 = df06_07.sort_values(['Position', 'Scorer_Rating'],ascending=[False,False])

PG06_07 = df06_07v2[df06_07.Position == 'PG']
SG06_07 = df06_07v2[df06_07.Position == 'SG']
SF06_07 = df06_07v2[df06_07.Position == 'SF']
PF06_07 = df06_07v2[df06_07.Position == 'PF']
C06_07 = df06_07v2[df06_07.Position == 'C']

PG06_07_starters = df06_07v2[df06_07.Position == 'PG'].head(1)
SG06_07_starters = df06_07v2[df06_07.Position == 'SG'].head(1)
SF06_07_starters = df06_07v2[df06_07.Position == 'SF'].head(1)
PF06_07_starters = df06_07v2[df06_07.Position == 'PF'].head(1)
C06_07_starters = df06_07v2[df06_07.Position == 'C'].head(1)

SR06_07 = pd.concat([PG06_07, SG06_07, SF06_07, PF06_07, C06_07])
SR06_07_starters = pd.concat([PG06_07_starters, SG06_07_starters, SF06_07_starters, PF06_07_starters, C06_07_starters])
#---------------------------------------------------------------------------------------------------------------------#

**2007-08 Scorer Rating**

df07_08 = NBA2000_2016v4[NBA2000_2016v4.Season == 2008]
df07_08.sort_values(by='Scorer_Rating', ascending=False)
df07_08v2 = df07_08.sort_values(['Position', 'Scorer_Rating'],ascending=[False,False])

PG07_08 = df07_08v2[df07_08.Position == 'PG']
SG07_08 = df07_08v2[df07_08.Position == 'SG']
SF07_08 = df07_08v2[df07_08.Position == 'SF']
PF07_08 = df07_08v2[df07_08.Position == 'PF']
C07_08 = df07_08v2[df07_08.Position == 'C']

PG07_08_starters = df07_08v2[df07_08.Position == 'PG'].head(1)
SG07_08_starters = df07_08v2[df07_08.Position == 'SG'].head(1)
SF07_08_starters = df07_08v2[df07_08.Position == 'SF'].head(1)
PF07_08_starters = df07_08v2[df07_08.Position == 'PF'].head(1)
C07_08_starters = df07_08v2[df07_08.Position == 'C'].head(1)

SR07_08 = pd.concat([PG07_08, SG07_08, SF07_08, PF07_08, C07_08])
SR07_08_starters = pd.concat([PG07_08_starters, SG07_08_starters, SF07_08_starters, PF07_08_starters, C07_08_starters])
#---------------------------------------------------------------------------------------------------------------------#

**2008-09 Scorer Rating**

df08_09 = NBA2000_2016v4[NBA2000_2016v4.Season == 2009]
df08_09.sort_values(by='Scorer_Rating', ascending=False)

df08_09v2 = df08_09.sort_values(['Position', 'Scorer_Rating'],ascending=[False,False])

PG08_09 = df08_09v2[df08_09.Position == 'PG']
SG08_09 = df08_09v2[df08_09.Position == 'SG']
SF08_09 = df08_09v2[df08_09.Position == 'SF']
PF08_09 = df08_09v2[df08_09.Position == 'PF']
C08_09 = df08_09v2[df08_09.Position == 'C']

PG08_09_starters = df08_09v2[df08_09.Position == 'PG'].head(1)
SG08_09_starters = df08_09v2[df08_09.Position == 'SG'].head(1)
SF08_09_starters = df08_09v2[df08_09.Position == 'SF'].head(1)
PF08_09_starters = df08_09v2[df08_09.Position == 'PF'].head(1)
C08_09_starters = df08_09v2[df08_09.Position == 'C'].head(1)

SR08_09 = pd.concat([PG08_09, SG08_09, SF08_09, PF08_09, C08_09])
SR08_09_starters = pd.concat([PG08_09_starters, SG08_09_starters, SF08_09_starters, PF08_09_starters, C08_09_starters])
#---------------------------------------------------------------------------------------------------------------------#

**2009-2010 Scorer Rating**

df09_10 = NBA2000_2016v4[NBA2000_2016v4.Season == 2010]
df09_10.sort_values(by='Scorer_Rating', ascending=False)
df09_10v2 = df09_10.sort_values(['Position', 'Scorer_Rating'],ascending=[False,False])

PG09_10 = df09_10v2[df09_10.Position == 'PG']
SG09_10 = df09_10v2[df09_10.Position == 'SG']
SF09_10 = df09_10v2[df09_10.Position == 'SF']
PF09_10 = df09_10v2[df09_10.Position == 'PF']
C09_10 = df09_10v2[df09_10.Position == 'C']

PG09_10_starters = df09_10v2[df09_10.Position == 'PG'].head(1)
SG09_10_starters = df09_10v2[df09_10.Position == 'SG'].head(1)
SF09_10_starters = df09_10v2[df09_10.Position == 'SF'].head(1)
PF09_10_starters = df09_10v2[df09_10.Position == 'PF'].head(1)
C09_10_starters = df09_10v2[df09_10.Position == 'C'].head(1)


SR09_10 = pd.concat([PG09_10, SG09_10, SF09_10, PF09_10, C09_10])
SR09_10_starters = pd.concat([PG09_10_starters, SG09_10_starters, SF09_10_starters, PF09_10_starters, C09_10_starters])

#_____________________________________________________________________________________________________________________#
#---------------------------------------------------------------------------------------------------------------------#

**2010-11 Scorer Rating**

df10_11 = NBA2000_2016v4[NBA2000_2016v4.Season == 2011]
df10_11.sort_values(by='Scorer_Rating', ascending=False)
df10_11v2 = df10_11.sort_values(['Position', 'Scorer_Rating'],ascending=[False,False])

PG10_11 = df10_11v2[df10_11.Position == 'PG']
SG10_11 = df10_11v2[df10_11.Position == 'SG']
SF10_11 = df10_11v2[df10_11.Position == 'SF']
PF10_11 = df10_11v2[df10_11.Position == 'PF']
C10_11 = df10_11v2[df10_11.Position == 'C']

PG10_11_starters = df10_11v2[df10_11.Position == 'PG'].head(1)
SG10_11_starters = df10_11v2[df10_11.Position == 'SG'].head(1)
SF10_11_starters = df10_11v2[df10_11.Position == 'SF'].head(1)
PF10_11_starters = df10_11v2[df10_11.Position == 'PF'].head(1)
C10_11_starters = df10_11v2[df10_11.Position == 'C'].head(1)

SR10_11 = pd.concat([PG10_11, SG10_11, SF10_11, PF10_11, C10_11])

SR10_11_starters = pd.concat([PG10_11_starters, SG10_11_starters, SF10_11_starters, PF10_11_starters, C10_11_starters])
#---------------------------------------------------------------------------------------------------------------------#

**2011-12 Scorer Rating**

df11_12 = NBA2000_2016v4[NBA2000_2016v4.Season == 2012]
df11_12.sort_values(by='Scorer_Rating', ascending=False)
df11_12v2 = df11_12.sort_values(['Position', 'Scorer_Rating'],ascending=[False,False])

PG11_12 = df11_12v2[df11_12.Position == 'PG']
SG11_12 = df11_12v2[df11_12.Position == 'SG']
SF11_12 = df11_12v2[df11_12.Position == 'SF']
PF11_12 = df11_12v2[df11_12.Position == 'PF']
C11_12 = df11_12v2[df11_12.Position == 'C']

PG11_12_starters = df11_12v2[df11_12.Position == 'PG'].head(1)
SG11_12_starters = df11_12v2[df11_12.Position == 'SG'].head(1)
SF11_12_starters = df11_12v2[df11_12.Position == 'SF'].head(1)
PF11_12_starters = df11_12v2[df11_12.Position == 'PF'].head(1)
C11_12_starters = df11_12v2[df11_12.Position == 'C'].head(1)

SR11_12 = pd.concat([PG11_12, SG11_12, SF11_12, PF11_12, C11_12])
SR11_12_starters = pd.concat([PG11_12_starters, SG11_12_starters, SF11_12_starters, PF11_12_starters, C11_12_starters])
#---------------------------------------------------------------------------------------------------------------------#

**2012-2013 Scorer Rating**

df12_13 = NBA2000_2016v4[NBA2000_2016v4.Season == 2013]
df12_13.sort_values(by='Scorer_Rating', ascending=False)
df12_13v2 = df12_13.sort_values(['Position', 'Scorer_Rating'],ascending=[False,False])

PG12_13 = df12_13v2[df12_13.Position == 'PG']
SG12_13 = df12_13v2[df12_13.Position == 'SG']
SF12_13 = df12_13v2[df12_13.Position == 'SF']
PF12_13 = df12_13v2[df12_13.Position == 'PF']
C12_13 = df12_13v2[df12_13.Position == 'C']

PG12_13_starters = df12_13v2[df12_13.Position == 'PG'].head(1)
SG12_13_starters = df12_13v2[df12_13.Position == 'SG'].head(1)
SF12_13_starters = df12_13v2[df12_13.Position == 'SF'].head(1)
PF12_13_starters = df12_13v2[df12_13.Position == 'PF'].head(1)
C12_13_starters = df12_13v2[df12_13.Position == 'C'].head(1)

SR12_13 = pd.concat([PG12_13, SG12_13, SF12_13, PF12_13, C12_13])
SR12_13_starters = pd.concat([PG12_13_starters, SG12_13_starters, SF12_13_starters, PF12_13_starters, C12_13_starters])
#---------------------------------------------------------------------------------------------------------------------#

**2013-2014 Scorer Rating**

df13_14 = NBA2000_2016v4[NBA2000_2016v4.Season == 2014]
df13_14.sort_values(by='Scorer_Rating', ascending=False)

df13_14v2 = df13_14.sort_values(['Position', 'Scorer_Rating'],ascending=[False,False])

PG13_14 = df13_14v2[df13_14.Position == 'PG']
SG13_14 = df13_14v2[df13_14.Position == 'SG']
SF13_14 = df13_14v2[df13_14.Position == 'SF']
PF13_14 = df13_14v2[df13_14.Position == 'PF']
C13_14 = df13_14v2[df13_14.Position == 'C']

PG13_14_starters = df13_14v2[df13_14.Position == 'PG'].head(1)
SG13_14_starters = df13_14v2[df13_14.Position == 'SG'].head(1)
SF13_14_starters = df13_14v2[df13_14.Position == 'SF'].head(1)
PF13_14_starters = df13_14v2[df13_14.Position == 'PF'].head(1)
C13_14_starters = df13_14v2[df13_14.Position == 'C'].head(1)

SR13_14 = pd.concat([PG13_14, SG13_14, SF13_14, PF13_14, C13_14])
SR13_14_starters = pd.concat([PG13_14_starters, SG13_14_starters, SF13_14_starters, PF13_14_starters, C13_14_starters])
#---------------------------------------------------------------------------------------------------------------------#

**2014-15 Scorer Rating**

df14_15 = NBA2000_2016v4[NBA2000_2016v4.Season == 2015]
df14_15.sort_values(by='Scorer_Rating', ascending=False)

df14_15v2 = df14_15.sort_values(['Position', 'Scorer_Rating'],ascending=[False,False])

PG14_15 = df14_15v2[df14_15.Position == 'PG']
SG14_15 = df14_15v2[df14_15.Position == 'SG']
SF14_15 = df14_15v2[df14_15.Position == 'SF']
PF14_15 = df14_15v2[df14_15.Position == 'PF']
C14_15 = df14_15v2[df14_15.Position == 'C']

PG14_15_starters = df14_15v2[df14_15.Position == 'PG'].head(1)
SG14_15_starters = df14_15v2[df14_15.Position == 'SG'].head(1)
SF14_15_starters = df14_15v2[df14_15.Position == 'SF'].head(1)
PF14_15_starters = df14_15v2[df14_15.Position == 'PF'].head(1)
C14_15_starters = df14_15v2[df14_15.Position == 'C'].head(1)

SR14_15 = pd.concat([PG14_15, SG14_15, SF14_15, PF14_15, C14_15])
SR14_15_starters = pd.concat([PG14_15_starters, SG14_15_starters, SF14_15_starters, PF14_15_starters, C14_15_starters])
#---------------------------------------------------------------------------------------------------------------------#

**2015-16 Scorer Rating**

df15_16 = NBA2000_2016v4[NBA2000_2016v4.Season == 2016]
df15_16.sort_values(by='Scorer_Rating', ascending=False)

df15_16v2 = df15_16.sort_values(['Position', 'Scorer_Rating'],ascending=[False,False])

PG15_16 = df15_16v2[df15_16.Position == 'PG']
SG15_16 = df15_16v2[df15_16.Position == 'SG']
SF15_16 = df15_16v2[df15_16.Position == 'SF']
PF15_16 = df15_16v2[df15_16.Position == 'PF']
C15_16 = df15_16v2[df15_16.Position == 'C']

PG15_16_starters = df15_16v2[df15_16.Position == 'PG'].head(1)
SG15_16_starters = df15_16v2[df15_16.Position == 'SG'].head(1)
SF15_16_starters = df15_16v2[df15_16.Position == 'SF'].head(1)
PF15_16_starters = df15_16v2[df15_16.Position == 'PF'].head(1)
C15_16_starters = df15_16v2[df15_16.Position == 'C'].head(1)

SR15_16 = pd.concat([PG15_16, SG15_16, SF15_16, PF15_16, C15_16])
SR15_16_starters = pd.concat([PG15_16_starters, SG15_16_starters, SF15_16_starters, PF15_16_starters, C15_16_starters])

## Playmaker Rating

**2000-01 Playmaker Rating**

df00_01 = NBA2000_2016v7[NBA2000_2016v7.Season == 2001]
df00_01.sort_values(by='Playmaker_Rating', ascending=False)
df00_01v2 = df00_01.sort_values(['Position', 'Playmaker_Rating'],ascending=[False,False])

PG00_01 = df00_01v2[df00_01.Position == 'PG']
SG00_01 = df00_01v2[df00_01.Position == 'SG']
SF00_01 = df00_01v2[df00_01.Position == 'SF']
PF00_01 = df00_01v2[df00_01.Position == 'PF']
C00_01 = df00_01v2[df00_01.Position == 'C']

PG00_01_starters = df00_01v2[df00_01.Position == 'PG'].head(1)
SG00_01_starters = df00_01v2[df00_01.Position == 'SG'].head(1)
SF00_01_starters = df00_01v2[df00_01.Position == 'SF'].head(1)
PF00_01_starters = df00_01v2[df00_01.Position == 'PF'].head(1)
C00_01_starters = df00_01v2[df00_01.Position == 'C'].head(1)

PR00_01 = pd.concat([PG00_01, SG00_01, SF00_01, PF00_01, C00_01])
PR00_01_starters = pd.concat([PG00_01_starters, SG00_01_starters, SF00_01_starters, PF00_01_starters, C00_01_starters])
#---------------------------------------------------------------------------------------------------------------------#

**2001-02 Playmaker Rating**

df01_02 = NBA2000_2016v7[NBA2000_2016v7.Season == 2002]
df01_02.sort_values(by='Playmaker_Rating', ascending=False)
df01_02v2 = df01_02.sort_values(['Position', 'Playmaker_Rating'],ascending=[False,False])

PG01_02 = df01_02v2[df01_02.Position == 'PG']
SG01_02 = df01_02v2[df01_02.Position == 'SG']
SF01_02 = df01_02v2[df01_02.Position == 'SF']
PF01_02 = df01_02v2[df01_02.Position == 'PF']
C01_02 = df01_02v2[df01_02.Position == 'C']

PG01_02_starters = df01_02v2[df01_02.Position == 'PG'].head(1)
SG01_02_starters = df01_02v2[df01_02.Position == 'SG'].head(1)
SF01_02_starters = df01_02v2[df01_02.Position == 'SF'].head(1)
PF01_02_starters = df01_02v2[df01_02.Position == 'PF'].head(1)
C01_02_starters = df01_02v2[df01_02.Position == 'C'].head(1)

PR01_02 = pd.concat([PG01_02, SG01_02, SF01_02, PF01_02, C01_02])
PR01_02_starters = pd.concat([PG01_02_starters, SG01_02_starters, SF01_02_starters, PF01_02_starters, C01_02_starters])
#---------------------------------------------------------------------------------------------------------------------#

**2002-03 Playmaker Rating**

df02_03 = NBA2000_2016v7[NBA2000_2016v7.Season == 2003]
df02_03.sort_values(by='Playmaker_Rating', ascending=False)

df02_03v2 = df02_03.sort_values(['Position', 'Playmaker_Rating'],ascending=[False,False])

PG02_03 = df02_03v2[df02_03.Position == 'PG']
SG02_03 = df02_03v2[df02_03.Position == 'SG']
SF02_03 = df02_03v2[df02_03.Position == 'SF']
PF02_03 = df02_03v2[df02_03.Position == 'PF']
C02_03 = df02_03v2[df02_03.Position == 'C']

PG02_03_starters = df02_03v2[df02_03.Position == 'PG'].head(1)
SG02_03_starters = df02_03v2[df02_03.Position == 'SG'].head(1)
SF02_03_starters = df02_03v2[df02_03.Position == 'SF'].head(1)
PF02_03_starters = df02_03v2[df02_03.Position == 'PF'].head(1)
C02_03_starters = df02_03v2[df02_03.Position == 'C'].head(1)

PR02_03 = pd.concat([PG02_03, SG02_03, SF02_03, PF02_03, C02_03])
PR02_03_starters = pd.concat([PG02_03_starters, SG02_03_starters, SF02_03_starters, PF02_03_starters, C02_03_starters])
#---------------------------------------------------------------------------------------------------------------------#

**2003-2004 Playmaker Rating**
df03_04 = NBA2000_2016v7[NBA2000_2016v7.Season == 2004]
df03_04.sort_values(by='Playmaker_Rating', ascending=False)
df03_04v2 = df03_04.sort_values(['Position', 'Playmaker_Rating'],ascending=[False,False])

PG03_04 = df03_04v2[df03_04.Position == 'PG']
SG03_04 = df03_04v2[df03_04.Position == 'SG']
SF03_04 = df03_04v2[df03_04.Position == 'SF']
PF03_04 = df03_04v2[df03_04.Position == 'PF']
C03_04 = df03_04v2[df03_04.Position == 'C']

PG03_04_starters = df03_04v2[df03_04.Position == 'PG'].head(1)
SG03_04_starters = df03_04v2[df03_04.Position == 'SG'].head(1)
SF03_04_starters = df03_04v2[df03_04.Position == 'SF'].head(1)
PF03_04_starters = df03_04v2[df03_04.Position == 'PF'].head(1)
C03_04_starters = df03_04v2[df03_04.Position == 'C'].head(1)


PR03_04 = pd.concat([PG03_04, SG03_04, SF03_04, PF03_04, C03_04])
PR03_04_starters = pd.concat([PG03_04_starters, SG03_04_starters, SF03_04_starters, PF03_04_starters, C03_04_starters])
#---------------------------------------------------------------------------------------------------------------------#

**2004-2005 Playmaker Rating**

df04_05 = NBA2000_2016v7[NBA2000_2016v7.Season == 2005]
df04_05.sort_values(by='Playmaker_Rating', ascending=False)

df04_05v2 = df04_05.sort_values(['Position', 'Playmaker_Rating'],ascending=[False,False])

PG04_05 = df04_05v2[df04_05.Position == 'PG']
SG04_05 = df04_05v2[df04_05.Position == 'SG']
SF04_05 = df04_05v2[df04_05.Position == 'SF']
PF04_05 = df04_05v2[df04_05.Position == 'PF']
C04_05 = df04_05v2[df04_05.Position == 'C']

PG04_05_starters = df04_05v2[df04_05.Position == 'PG'].head(1)
SG04_05_starters = df04_05v2[df04_05.Position == 'SG'].head(1)
SF04_05_starters = df04_05v2[df04_05.Position == 'SF'].head(1)
PF04_05_starters = df04_05v2[df04_05.Position == 'PF'].head(1)
C04_05_starters = df04_05v2[df04_05.Position == 'C'].head(1)


PR04_05 = pd.concat([PG04_05, SG04_05, SF04_05, PF04_05, C04_05])
PR04_05_starters = pd.concat([PG04_05_starters, SG04_05_starters, SF04_05_starters, PF04_05_starters, C04_05_starters])
#---------------------------------------------------------------------------------------------------------------------#

**2005-06 Playmaker Rating**

df05_06 = NBA2000_2016v7[NBA2000_2016v7.Season == 2006]
df05_06.sort_values(by='Playmaker_Rating', ascending=False)

df05_06v2 = df05_06.sort_values(['Position', 'Playmaker_Rating'],ascending=[False,False])

PG05_06 = df05_06v2[df05_06.Position == 'PG']
SG05_06 = df05_06v2[df05_06.Position == 'SG']
SF05_06 = df05_06v2[df05_06.Position == 'SF']
PF05_06 = df05_06v2[df05_06.Position == 'PF']
C05_06 = df05_06v2[df05_06.Position == 'C']

PG05_06_starters = df05_06v2[df05_06.Position == 'PG'].head(1)
SG05_06_starters = df05_06v2[df05_06.Position == 'SG'].head(1)
SF05_06_starters = df05_06v2[df05_06.Position == 'SF'].head(1)
PF05_06_starters = df05_06v2[df05_06.Position == 'PF'].head(1)
C05_06_starters = df05_06v2[df05_06.Position == 'C'].head(1)

PR05_06 = pd.concat([PG05_06, SG05_06, SF05_06, PF05_06, C05_06])
PR05_06_starters = pd.concat([PG05_06_starters, SG05_06_starters, SF05_06_starters, PF05_06_starters, C05_06_starters])
#---------------------------------------------------------------------------------------------------------------------#

**2006-07 Scorer Rating**

df06_07 = NBA2000_2016v7[NBA2000_2016v7.Season == 2007]
df06_07.sort_values(by='Playmaker_Rating', ascending=False)

df06_07v2 = df06_07.sort_values(['Position', 'Playmaker_Rating'],ascending=[False,False])

PG06_07 = df06_07v2[df06_07.Position == 'PG']
SG06_07 = df06_07v2[df06_07.Position == 'SG']
SF06_07 = df06_07v2[df06_07.Position == 'SF']
PF06_07 = df06_07v2[df06_07.Position == 'PF']
C06_07 = df06_07v2[df06_07.Position == 'C']

PG06_07_starters = df06_07v2[df06_07.Position == 'PG'].head(1)
SG06_07_starters = df06_07v2[df06_07.Position == 'SG'].head(1)
SF06_07_starters = df06_07v2[df06_07.Position == 'SF'].head(1)
PF06_07_starters = df06_07v2[df06_07.Position == 'PF'].head(1)
C06_07_starters = df06_07v2[df06_07.Position == 'C'].head(1)

PR06_07 = pd.concat([PG06_07, SG06_07, SF06_07, PF06_07, C06_07])
PR06_07_starters = pd.concat([PG06_07_starters, SG06_07_starters, SF06_07_starters, PF06_07_starters, C06_07_starters])
#---------------------------------------------------------------------------------------------------------------------#

**2007-08 Playmaker Rating**

df07_08 = NBA2000_2016v7[NBA2000_2016v7.Season == 2008]
df07_08.sort_values(by='Playmaker_Rating', ascending=False)
df07_08v2 = df07_08.sort_values(['Position', 'Playmaker_Rating'],ascending=[False,False])

PG07_08 = df07_08v2[df07_08.Position == 'PG']
SG07_08 = df07_08v2[df07_08.Position == 'SG']
SF07_08 = df07_08v2[df07_08.Position == 'SF']
PF07_08 = df07_08v2[df07_08.Position == 'PF']
C07_08 = df07_08v2[df07_08.Position == 'C']

PG07_08_starters = df07_08v2[df07_08.Position == 'PG'].head(1)
SG07_08_starters = df07_08v2[df07_08.Position == 'SG'].head(1)
SF07_08_starters = df07_08v2[df07_08.Position == 'SF'].head(1)
PF07_08_starters = df07_08v2[df07_08.Position == 'PF'].head(1)
C07_08_starters = df07_08v2[df07_08.Position == 'C'].head(1)

PR07_08 = pd.concat([PG07_08, SG07_08, SF07_08, PF07_08, C07_08])
PR07_08_starters = pd.concat([PG07_08_starters, SG07_08_starters, SF07_08_starters, PF07_08_starters, C07_08_starters])
#---------------------------------------------------------------------------------------------------------------------#

**2008-09 Playmaker Rating**

df08_09 = NBA2000_2016v7[NBA2000_2016v7.Season == 2009]
df08_09.sort_values(by='Playmaker_Rating', ascending=False)

df08_09v2 = df08_09.sort_values(['Position', 'Playmaker_Rating'],ascending=[False,False])

PG08_09 = df08_09v2[df08_09.Position == 'PG']
SG08_09 = df08_09v2[df08_09.Position == 'SG']
SF08_09 = df08_09v2[df08_09.Position == 'SF']
PF08_09 = df08_09v2[df08_09.Position == 'PF']
C08_09 = df08_09v2[df08_09.Position == 'C']

PG08_09_starters = df08_09v2[df08_09.Position == 'PG'].head(1)
SG08_09_starters = df08_09v2[df08_09.Position == 'SG'].head(1)
SF08_09_starters = df08_09v2[df08_09.Position == 'SF'].head(1)
PF08_09_starters = df08_09v2[df08_09.Position == 'PF'].head(1)
C08_09_starters = df08_09v2[df08_09.Position == 'C'].head(1)

PR08_09 = pd.concat([PG08_09, SG08_09, SF08_09, PF08_09, C08_09])
PR08_09_starters = pd.concat([PG08_09_starters, SG08_09_starters, SF08_09_starters, PF08_09_starters, C08_09_starters])
#---------------------------------------------------------------------------------------------------------------------#

**2009-2010 Playmaker Rating**
df09_10 = NBA2000_2016v7[NBA2000_2016v7.Season == 2010]
df09_10.sort_values(by='Playmaker_Rating', ascending=False)
df09_10v2 = df09_10.sort_values(['Position', 'Playmaker_Rating'],ascending=[False,False])

PG09_10 = df09_10v2[df09_10.Position == 'PG']
SG09_10 = df09_10v2[df09_10.Position == 'SG']
SF09_10 = df09_10v2[df09_10.Position == 'SF']
PF09_10 = df09_10v2[df09_10.Position == 'PF']
C09_10 = df09_10v2[df09_10.Position == 'C']

PG09_10_starters = df09_10v2[df09_10.Position == 'PG'].head(1)
SG09_10_starters = df09_10v2[df09_10.Position == 'SG'].head(1)
SF09_10_starters = df09_10v2[df09_10.Position == 'SF'].head(1)
PF09_10_starters = df09_10v2[df09_10.Position == 'PF'].head(1)
C09_10_starters = df09_10v2[df09_10.Position == 'C'].head(1)


PR09_10 = pd.concat([PG09_10, SG09_10, SF09_10, PF09_10, C09_10])
PR09_10_starters = pd.concat([PG09_10_starters, SG09_10_starters, SF09_10_starters, PF09_10_starters, C09_10_starters])
#---------------------------------------------------------------------------------------------------------------------#

**2010-11 Playmaker Rating**

df10_11 = NBA2000_2016v7[NBA2000_2016v7.Season == 2011]
df10_11.sort_values(by='Playmaker_Rating', ascending=False)
df10_11v2 = df10_11.sort_values(['Position', 'Playmaker_Rating'],ascending=[False,False])

PG10_11 = df10_11v2[df10_11.Position == 'PG']
SG10_11 = df10_11v2[df10_11.Position == 'SG']
SF10_11 = df10_11v2[df10_11.Position == 'SF']
PF10_11 = df10_11v2[df10_11.Position == 'PF']
C10_11 = df10_11v2[df10_11.Position == 'C']

PG10_11_starters = df10_11v2[df10_11.Position == 'PG'].head(1)
SG10_11_starters = df10_11v2[df10_11.Position == 'SG'].head(1)
SF10_11_starters = df10_11v2[df10_11.Position == 'SF'].head(1)
PF10_11_starters = df10_11v2[df10_11.Position == 'PF'].head(1)
C10_11_starters = df10_11v2[df10_11.Position == 'C'].head(1)

PR10_11 = pd.concat([PG10_11, SG10_11, SF10_11, PF10_11, C10_11])
PR10_11_starters = pd.concat([PG10_11_starters, SG10_11_starters, SF10_11_starters, PF10_11_starters, C10_11_starters])
#---------------------------------------------------------------------------------------------------------------------#

**2011-12 Playmaker Rating**

df11_12 = NBA2000_2016v7[NBA2000_2016v7.Season == 2012]
df11_12.sort_values(by='Playmaker_Rating', ascending=False)

df11_12v2 = df11_12.sort_values(['Position', 'Playmaker_Rating'],ascending=[False,False])

PG11_12 = df11_12v2[df11_12.Position == 'PG']
SG11_12 = df11_12v2[df11_12.Position == 'SG']
SF11_12 = df11_12v2[df11_12.Position == 'SF']
PF11_12 = df11_12v2[df11_12.Position == 'PF']
C11_12 = df11_12v2[df11_12.Position == 'C']

PG11_12_starters = df11_12v2[df11_12.Position == 'PG'].head(1)
SG11_12_starters = df11_12v2[df11_12.Position == 'SG'].head(1)
SF11_12_starters = df11_12v2[df11_12.Position == 'SF'].head(1)
PF11_12_starters = df11_12v2[df11_12.Position == 'PF'].head(1)
C11_12_starters = df11_12v2[df11_12.Position == 'C'].head(1)

PR11_12 = pd.concat([PG11_12, SG11_12, SF11_12, PF11_12, C11_12])
PR11_12_starters = pd.concat([PG11_12_starters, SG11_12_starters, SF11_12_starters, PF11_12_starters, C11_12_starters])
#---------------------------------------------------------------------------------------------------------------------#

**2012-2013 Playmaker Rating**

df12_13 = NBA2000_2016v7[NBA2000_2016v7.Season == 2013]
df12_13.sort_values(by='Playmaker_Rating', ascending=False)
df12_13v2 = df12_13.sort_values(['Position', 'Playmaker_Rating'],ascending=[False,False])

PG12_13 = df12_13v2[df12_13.Position == 'PG']
SG12_13 = df12_13v2[df12_13.Position == 'SG']
SF12_13 = df12_13v2[df12_13.Position == 'SF']
PF12_13 = df12_13v2[df12_13.Position == 'PF']
C12_13 = df12_13v2[df12_13.Position == 'C']

PG12_13_starters = df12_13v2[df12_13.Position == 'PG'].head(1)
SG12_13_starters = df12_13v2[df12_13.Position == 'SG'].head(1)
SF12_13_starters = df12_13v2[df12_13.Position == 'SF'].head(1)
PF12_13_starters = df12_13v2[df12_13.Position == 'PF'].head(1)
C12_13_starters = df12_13v2[df12_13.Position == 'C'].head(1)

PR12_13 = pd.concat([PG12_13, SG12_13, SF12_13, PF12_13, C12_13])
PR12_13_starters = pd.concat([PG12_13_starters, SG12_13_starters, SF12_13_starters, PF12_13_starters, C12_13_starters])
#---------------------------------------------------------------------------------------------------------------------#

**2013-2014 Playmaker Rating**

df13_14 = NBA2000_2016v7[NBA2000_2016v7.Season == 2014]
df13_14.sort_values(by='Playmaker_Rating', ascending=False)

df13_14v2 = df13_14.sort_values(['Position', 'Playmaker_Rating'],ascending=[False,False])

PG13_14 = df13_14v2[df13_14.Position == 'PG']
SG13_14 = df13_14v2[df13_14.Position == 'SG']
SF13_14 = df13_14v2[df13_14.Position == 'SF']
PF13_14 = df13_14v2[df13_14.Position == 'PF']
C13_14 = df13_14v2[df13_14.Position == 'C']

PG13_14_starters = df13_14v2[df13_14.Position == 'PG'].head(1)
SG13_14_starters = df13_14v2[df13_14.Position == 'SG'].head(1)
SF13_14_starters = df13_14v2[df13_14.Position == 'SF'].head(1)
PF13_14_starters = df13_14v2[df13_14.Position == 'PF'].head(1)
C13_14_starters = df13_14v2[df13_14.Position == 'C'].head(1)

PR13_14 = pd.concat([PG13_14, SG13_14, SF13_14, PF13_14, C13_14])
PR13_14_starters = pd.concat([PG13_14_starters, SG13_14_starters, SF13_14_starters, PF13_14_starters, C13_14_starters])
#---------------------------------------------------------------------------------------------------------------------#

**2014-15 Playmaker Rating**

df14_15 = NBA2000_2016v7[NBA2000_2016v7.Season == 2015]
df14_15.sort_values(by='Playmaker_Rating', ascending=False)

df14_15v2 = df14_15.sort_values(['Position', 'Playmaker_Rating'],ascending=[False,False])

PG14_15 = df14_15v2[df14_15.Position == 'PG']
SG14_15 = df14_15v2[df14_15.Position == 'SG']
SF14_15 = df14_15v2[df14_15.Position == 'SF']
PF14_15 = df14_15v2[df14_15.Position == 'PF']
C14_15 = df14_15v2[df14_15.Position == 'C']

PG14_15_starters = df14_15v2[df14_15.Position == 'PG'].head(1)
SG14_15_starters = df14_15v2[df14_15.Position == 'SG'].head(1)
SF14_15_starters = df14_15v2[df14_15.Position == 'SF'].head(1)
PF14_15_starters = df14_15v2[df14_15.Position == 'PF'].head(1)
C14_15_starters = df14_15v2[df14_15.Position == 'C'].head(1)

PR14_15 = pd.concat([PG14_15, SG14_15, SF14_15, PF14_15, C14_15])
PR14_15_starters = pd.concat([PG14_15_starters, SG14_15_starters, SF14_15_starters, PF14_15_starters, C14_15_starters])
#---------------------------------------------------------------------------------------------------------------------#

**2015-16 Playmaker Rating**

df15_16 = NBA2000_2016v7[NBA2000_2016v7.Season == 2016]
df15_16.sort_values(by='Playmaker_Rating', ascending=False)

df15_16v2 = df15_16.sort_values(['Position', 'Playmaker_Rating'],ascending=[False,False])

PG15_16 = df15_16v2[df15_16.Position == 'PG']
SG15_16 = df15_16v2[df15_16.Position == 'SG']
SF15_16 = df15_16v2[df15_16.Position == 'SF']
PF15_16 = df15_16v2[df15_16.Position == 'PF']
C15_16 = df15_16v2[df15_16.Position == 'C']

PG15_16_starters = df15_16v2[df15_16.Position == 'PG'].head(1)
SG15_16_starters = df15_16v2[df15_16.Position == 'SG'].head(1)
SF15_16_starters = df15_16v2[df15_16.Position == 'SF'].head(1)
PF15_16_starters = df15_16v2[df15_16.Position == 'PF'].head(1)
C15_16_starters = df15_16v2[df15_16.Position == 'C'].head(1)

PR15_16 = pd.concat([PG15_16, SG15_16, SF15_16, PF15_16, C15_16])

PR15_16_starters = pd.concat([PG15_16_starters, SG15_16_starters, SF15_16_starters, PF15_16_starters, C15_16_starters])

### Preliminary & Exploratory Analysis

Thus far, my analysis has included plotting visualizations in order to better understand the data. Thus far, I have found that there is an overall positive correlation to many of the important player stats and to the Playmaker and Scorer Ratings. From 2000-2016 Playmaker Ratings appear to have remained the same. However, Scorer Ratings appear to have fluctuated over the last 16 years.

![Playmaker Rating 2000_2016](Playmaker Rating 2000_2016.png)

![Scorer Rating](Scorer Rating.png)

In regard to the Scorer and Playmaker Rating Distribution by Positions, both seem to have little variation among the positions, but Point Guards appear to lead in both types of ratings.

![Scorer Rating Positions](Scorer Rating Positions.png)

![Playmaker Rating by Position](Playmaker Rating by Position.png)

At this point, I plan to focus my analysis on accuracy of Playmaker and Scorer Ratings and how they relate to actual All-NBA teams in the past, using confidence intervals to select the average score rating cut-off point. 
I will also build a a few models with deep learning to make various predictions as well.
To begin, I created new a new dataset containing all players with the original variables and whether that player had made the All-NBA team (Yes/No) in which I then will build regression models and use clustering, bayesian stats and SVM to help predict who should be this years All-NBA Team. I will then compare to my original proposal of matching with the top Playmaker and Scorer Ratings Players.
